# Stock Analysis Agents
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MervinPraison/PraisonAI/blob/main/cookbooks/notebooks/stock_analysis_agents.ipynb)

## Dependencies

In [53]:
# Install dependencies without output
%pip install langchain_community > /dev/null
%pip install praisonai[crewai] > /dev/null
%pip install duckduckgo-search > /dev/null
%pip install sec-api > /dev/null
%pip install embedchain > /dev/null
%pip install html2text > /dev/null

## Tools

In [67]:
#ToDo: Add fix for the Output Error regarding BaseModel
import os
import re
import requests
import html2text
from bs4 import BeautifulSoup
from typing import Any, Optional, Type
from pydantic import BaseModel, Field
from praisonai_tools import BaseTool
from sec_api import QueryApi
from embedchain.models.data_type import DataType
from langchain.tools import tool


class FixedSEC10KToolSchema(BaseModel):
    """Input schema for SEC10KTool."""
    search_query: str = Field(
        ..., description="The query string to search within the 10-K report content."
    )


class SEC10KTool(BaseTool):
    name: str = "SEC10KTool"
    description: str = "Fetches and searches through the latest 10-K form content for a specified stock ticker."
    args_schema: Type[BaseModel] = FixedSEC10KToolSchema

    def __init__(self, stock_name: Optional[str] = None, **kwargs):
        super().__init__(**kwargs)
        if stock_name:
            content = self.get_10k_url_content(stock_name)
            if content:
                self.add(content)
                self.description = f"Search within {stock_name}'s latest 10-K form content."

    def get_10k_url_content(self, stock_name: str) -> Optional[str]:
        """Fetches the latest 10-K form for the specified stock ticker."""
        try:
            queryApi = QueryApi(api_key=os.environ['SEC_API_API_KEY'])
            query = {
                "query": {"query_string": {"query": f"ticker:{stock_name} AND formType:\"10-K\""}},
                "from": "0",
                "size": "1",
                "sort": [{"filedAt": {"order": "desc"}}]
            }
            filings = queryApi.get_filings(query)['filings']
            if not filings:
                return None

            url = filings[0]['linkToFilingDetails']
            headers = {"User-Agent": "crewai.com", "Accept-Encoding": "gzip, deflate"}
            response = requests.get(url, headers=headers)
            response.raise_for_status()

            text = html2text.HTML2Text().handle(response.text)
            return re.sub(r"[^a-zA-Z$0-9\s\n]", "", text)
        except requests.exceptions.RequestException as e:
            return f"Failed to fetch 10-K form: {e}"
        except Exception as e:
            return f"Error processing 10-K form content: {e}"

    def add(self, *args: Any, **kwargs: Any) -> None:
        kwargs["data_type"] = DataType.TEXT
        super().add(*args, **kwargs)

    def _run(self, search_query: str) -> str:
        """Searches through the 10-K form content."""
        return super()._run(query=search_query)


class FixedSEC10QToolSchema(BaseModel):
    """Input schema for SEC10QTool."""
    search_query: str = Field(
        ..., description="The query string to search within the 10-Q report content."
    )


class SEC10QTool(BaseTool):
    name: str = "SEC10QTool"
    description: str = "Fetches and searches through the latest 10-Q form content for a specified stock ticker."
    args_schema: Type[BaseModel] = FixedSEC10QToolSchema

    def __init__(self, stock_name: Optional[str] = None, **kwargs):
        super().__init__(**kwargs)
        if stock_name:
            content = self.get_10q_url_content(stock_name)
            if content:
                self.add(content)
                self.description = f"Search within {stock_name}'s latest 10-Q form content."

    def get_10q_url_content(self, stock_name: str) -> Optional[str]:
        """Fetches the latest 10-Q form for the specified stock ticker."""
        try:
            queryApi = QueryApi(api_key=os.environ['SEC_API_API_KEY'])
            query = {
                "query": {"query_string": {"query": f"ticker:{stock_name} AND formType:\"10-Q\""}},
                "from": "0",
                "size": "1",
                "sort": [{"filedAt": {"order": "desc"}}]
            }
            filings = queryApi.get_filings(query)['filings']
            if not filings:
                return None

            url = filings[0]['linkToFilingDetails']
            headers = {"User-Agent": "crewai.com", "Accept-Encoding": "gzip, deflate"}
            response = requests.get(url, headers=headers)
            response.raise_for_status()

            text = html2text.HTML2Text().handle(response.text)
            return re.sub(r"[^a-zA-Z$0-9\s\n]", "", text)
        except requests.exceptions.RequestException as e:
            return f"Failed to fetch 10-Q form: {e}"
        except Exception as e:
            return f"Error processing 10-Q form content: {e}"

    def add(self, *args: Any, **kwargs: Any) -> None:
        kwargs["data_type"] = DataType.TEXT
        super().add(*args, **kwargs)

    def _run(self, search_query: str) -> str:
        """Searches through the 10-Q form content."""
        return super()._run(query=search_query)


class ScrapeWebsiteTool(BaseTool):
    name: str = "WebContentReaderTool"
    description: str = "Fetches and reads the main text content from a specified webpage URL."

    def _run(self, url: str) -> str:
        """Reads the content of a webpage and returns up to 5000 characters of text."""
        try:
            response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            text_content = soup.get_text(separator="\n", strip=True)
            return text_content[:5000]
        except requests.exceptions.RequestException as e:
            return f"Failed to retrieve content from {url}: {e}"


class CalculatorTool():
    @tool("Make a calculation")
    def calculate(operation: str) -> str:
        """Performs mathematical calculations. Accepts expressions like `200*7` or `5000/2*10`."""
        try:
            return str(eval(operation))
        except Exception as e:
            return f"Error: {e}"


## YAML Prompt

In [57]:
agent_yaml = """
framework: "crewai"
topic: "Automated Stock Analysis and Investment Recommendation"
roles:
  financial_analyst:
    role: "The Best Financial Analyst"
    backstory: |
      A seasoned financial analyst with deep expertise in stock market analysis and investment
      strategies, working to impress a highly important customer.
    goal: "Deliver impressive financial data and market trends analysis."
    tasks:
      financial_analysis:
        description: |
          Conduct a thorough analysis of NVIDIA's financial health and market performance.
          Examine key financial metrics such as P/E ratio, EPS growth, revenue trends, and debt-to-equity ratio.
          Analyze the stock's performance relative to its industry peers and overall market trends.
        expected_output: |
          The final report must expand on the initial summary, providing a clear assessment of the stock's financial standing,
          strengths and weaknesses, and comparison with competitors in the current market. Use the most recent data available.
    tools:
      - "ScrapeWebsiteTool"
      - "WebsiteSearchTool"
      - "CalculatorTool"
      - "SEC10QTool"
      - "SEC10KTool"

  research_analyst:
    role: "Staff Research Analyst"
    backstory: |
      Known as the top research analyst, skilled in interpreting data, tracking company news,
      and uncovering market sentiments. Working on a critical analysis for an important client.
    goal: "Excel in gathering and interpreting data to impress the customer."
    tasks:
      research:
        description: |
          Collect and summarize recent news articles, press releases, and market analyses related to NVIDIA
          and its industry. Pay attention to significant events, market sentiments, and analyst opinions.
          Include upcoming events like earnings and others.
        expected_output: |
          A report that includes a comprehensive summary of recent news, notable shifts in market sentiment,
          and potential impacts on the stock. Include the stock ticker as NVIDIA. Use the most current data available.
    tools:
      - "ScrapeWebsiteTool"
      - "SEC10QTool"
      - "SEC10KTool"

  investment_advisor:
    role: "Private Investment Advisor"
    backstory: |
      An experienced investment advisor who integrates various analytical insights to create strategic
      investment advice, aiming to impress a highly important client.
    goal: "Provide a comprehensive analysis and investment recommendation for the customer."
    tasks:
      recommend:
        description: |
          Review and synthesize analyses from the Financial Analyst and Research Analyst. Combine these insights
          into a comprehensive investment recommendation, considering all aspects, including financial health,
          market sentiment, and qualitative data from EDGAR filings. Include insider trading activity and upcoming events like earnings.
        expected_output: |
          The final recommendation must be a super detailed report, offering a clear investment stance and strategy with supporting evidence.
          Ensure it is well-formatted and visually appealing for the customer.
    tools:
      - "ScrapeWebsiteTool"
      - "WebsiteSearchTool"
      - "CalculatorTool"

dependencies: []
"""

## Main

In [66]:
import os
from praisonai import PraisonAI
from google.colab import userdata

# Create a PraisonAI instance with the agent_yaml content
praisonai = PraisonAI(agent_yaml=agent_yaml, tools=[ScrapeWebsiteTool, WebsiteSearchTool,
                                                    CalculatorTool, SEC10QTool, SEC10KTool])

# Add OPENAI_API_KEY Secrets to Google Colab on the Left Hand Side 🔑 or Enter Manually Below
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY') or "ENTER OPENAI_API_KEY HERE"
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

# Run PraisonAI
result = praisonai.run()

# Print the result
print(result) # 10/10


PydanticInvalidForJsonSchema: Cannot generate a JsonSchema for core_schema.CallableSchema

For further information visit https://errors.pydantic.dev/2.9/u/invalid-for-json-schema